<a href="https://colab.research.google.com/github/yjyjy131/ProblemSolving/blob/main/simulation/boj23288_%EC%A3%BC%EC%82%AC%EC%9C%84%EA%B5%B4%EB%A6%AC%EA%B8%B02_g3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.acmicpc.net/problem/23288

In [ ]:
# 지도의 좌표 (r, c) => (1, 1) 부터 시작, (n, m) 이 가장 끝
# r : 위쪽으로부터 떨어진 칸의 개수
# c : 서쪽으로부터 떨어진 칸의 개수
# 주사위가 (1, 1) 에 놓여져 있음
# 주사위의 처음 이동 방향은 동쪽
# 1. 이동방향으로 한칸 굴러간다. 칸이 없다면 이동 방향을 반대로 해서 한칸 구른다.
# 2. 주사위가 도착한 칸 (x, y) 에 대한 점수를 획득
#       ㄴ 점수 : (x, y) 의 정수가 B 일 때 (x, y) 에서 동서남북 방향으로 연속해서 이동 가능한 칸의 수 C
#                이 때, 이동 가능한 칸에는 모두 B 가 있어야 함
#                B x C 의 값이 점수
# 3. 주사위의 아랫면에 있는 정수 A 와 주사위가 있는 칸 (x, y) 의 정수 B 를 비교해 이동방향 결정
#       ㄴ A > B : 이동방향 90 도 시계방향 회전
#       ㄴ A < B : 이동방향 90 도 반시계방향 회전
#       ㄴ A = B : 변화 없음
#

from collections import deque
import copy


# 방향에 따라 주사위 굴리기
def move_dice(arr, dir):
    arr = copy.deepcopy(arr)

    if dir == 0:
        arr[1].appendleft(arr[3].pop())
        arr[3] = deque([arr[1].pop()])

    elif dir == 1:
        arr[1].append(arr[3].pop())
        arr[3] = deque([arr[1].popleft()])

    elif dir == 2:
        arr[2].append(arr[1][1])
        arr[3].append(arr[2].popleft())
        arr[0].append(arr[3].popleft())
        arr[1][1] = arr[0].popleft()

    elif dir == 3:
        arr[0].append(arr[1][1])
        arr[3].append(arr[0].popleft())
        arr[2].append(arr[3].popleft())
        arr[1][1] = arr[2].popleft()

    return arr


# 지도에서 연속으로 이동할 수 있는 칸 수 계산
def dfs(arr, x, y, val):
    global max_cnt
    max_cnt += 1
    go_x = [-1, 1, 0, 0]
    go_y = [0, 0, -1, 1]

    for gx, gy in zip(go_x, go_y):
        nx = x + gx
        ny = y + gy

        if 0 <= nx < n and 0 <= ny < m and not visited[nx][ny] and arr[nx][ny] == val:
            visited[nx][ny] = 1
            dfs(arr, nx, ny, val)


# 움직여야하는 방향과 방향에 따른 x, y 좌표값
def set_dir(x, y, dir):
    dx = [0, 0, 1, -1]
    dy = [1, -1, 0, 0]
    nx, ny = x + dx[dir], y + dy[dir]

    if 0 <= nx < n and 0 <= ny < m:
        nx, ny = x + dx[dir], y + dy[dir]
        return nx, ny, dir
    else:
        # 만약, 이동 방향에 칸이 없다면, 이동 방향을 반대로 한 다음 한 칸 굴러간다
        if dir == 0:
            dir = 1
        elif dir == 1:
            dir = 0
        elif dir == 2:
            dir = 3
        elif dir == 3:
            dir = 2
        nx, ny = x + dx[dir], y + dy[dir]
        return nx, ny, dir


if __name__ == '__main__':
    n, m, k = map(int, input().split())
    board = [list(map(int, input().split())) for _ in range(n)]
    dice = [deque([2]), deque([4, 1, 3]), deque([5]), deque([6])]
    dir, ans = 0, 0
    x, y = 0, 0
    clock = {0: 2, 2: 1, 1: 3, 3: 0}
    clock_wise = {0: 3, 3: 1, 1: 2, 2: 0}

    move_dice(dice, 0)
    move_dice(dice, 1)
    move_dice(dice, 2)
    move_dice(dice, 3)

    global max_cnt

    for _ in range(k):
        # 이동 방향으로 한칸 구른다
        x, y, dir = set_dir(x, y, dir)
        dice = move_dice(dice, dir)

        # 점수를 획득한다
        max_cnt = -1
        visited = [[0 for _ in range(m)] for _ in range(n)]
        dfs(board, x, y, board[x][y])
        score = board[x][y] * max_cnt
        if score == 0:
            score = board[x][y]

        ans += score

        # 방향을 바꾼다
        if dice[3][0] > board[x][y]:
            dir = clock.get(dir)
        elif dice[3][0] < board[x][y]:
            dir = clock_wise.get(dir)

    print(ans)
